<a href="https://colab.research.google.com/github/mvrcx/SWP_Blitzer/blob/main/FaceMask_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import dlib
from PIL import Image
import os 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
with open('/content/drive/MyDrive/SWP_Blitzer/landmark_grayscales_covered_uncovered.npy', 'rb') as f:
    data = np.load(f, allow_pickle=True)

In [7]:
len(data)

1065

In [8]:
np.random.shuffle(data)

In [ ]:
data

In [10]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(68, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

if torch.cuda.is_available():
    net.cuda()

Net(
  (fc1): Linear(in_features=68, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [13]:
optimizer = optim.Adam(net.parameters(), lr=0.0001)

def train(n_epochs, data):
    for epoch in range(n_epochs):
        for i, (X, y) in enumerate(data):

            input = torch.from_numpy(X).float()
            target = torch.from_numpy(y).float()
            if torch.cuda.is_available():
                input = input.cuda()
                target = target.cuda()

            output = net(input)
            
            criterion = nn.MSELoss()

            loss = criterion(output, target)

            # create your optimizer

            # in your training loop:
            optimizer.zero_grad()   # zero the gradient buffers
            loss.backward()
            optimizer.step()    # Does the update

            print("[Epoch %d/%d] [Batch %d/%d] [loss: %f]" % (epoch+1, n_epochs, i, len(data), loss.item()))
            

In [14]:
train(100, data)

[Epoch 1/100] [Batch 0/1065] [loss: 0.094435]
[Epoch 1/100] [Batch 1/1065] [loss: 0.096081]
[Epoch 1/100] [Batch 2/1065] [loss: 0.094563]
[Epoch 1/100] [Batch 3/1065] [loss: 0.093115]
[Epoch 1/100] [Batch 4/1065] [loss: 0.482497]
[Epoch 1/100] [Batch 5/1065] [loss: 0.092996]
[Epoch 1/100] [Batch 6/1065] [loss: 0.093541]
[Epoch 1/100] [Batch 7/1065] [loss: 0.091740]
[Epoch 1/100] [Batch 8/1065] [loss: 0.092144]
[Epoch 1/100] [Batch 9/1065] [loss: 0.091002]
[Epoch 1/100] [Batch 10/1065] [loss: 0.091085]
[Epoch 1/100] [Batch 11/1065] [loss: 0.490324]
[Epoch 1/100] [Batch 12/1065] [loss: 0.088592]
[Epoch 1/100] [Batch 13/1065] [loss: 0.089186]
[Epoch 1/100] [Batch 14/1065] [loss: 0.089597]
[Epoch 1/100] [Batch 15/1065] [loss: 0.494788]
[Epoch 1/100] [Batch 16/1065] [loss: 0.088687]
[Epoch 1/100] [Batch 17/1065] [loss: 0.089597]
[Epoch 1/100] [Batch 18/1065] [loss: 0.087537]
[Epoch 1/100] [Batch 19/1065] [loss: 0.088530]
[Epoch 1/100] [Batch 20/1065] [loss: 0.087893]
[Epoch 1/100] [Batch 21

KeyboardInterrupt: ignored

In [ ]:
score = 0
for i in range(len(X_train)):
    print(net(torch.from_numpy(X_train[i]).float().cuda()))
    print(y_train[i])

print(score/(len(X_train)+1))

In [ ]:
torch.save(net.state_dict(), "/content/drive/MyDrive/SWP_Blitzer/detect_covered_face.pt")